In [ ]:
import joblib
import numpy as np
from PIL import Image
from pathlib import Path
import cv2
import openflexure_microscope_client as ofm_client
import time

# ================================
# 1. Initialize microscope
# ================================
microscope = ofm_client.MicroscopeClient("172.20.69.53")

# ================================
# 2. Load model and scaler
# ================================
model_path = r"C:\Users\Ezekiel\Desktop\model trained\New folder\signed_step_size_model.pkl"
scaler_path = r"C:\Users\Ezekiel\Desktop\model trained\New folder\signed_step_scaler.pkl"
regressor = joblib.load(model_path)
scaler = joblib.load(scaler_path)

# ================================
# 3. Output directory
# ================================
output_dir = Path.home() / "Desktop" / "Random forest" / "model robteness1" / "variance1"
output_dir.mkdir(parents=True, exist_ok=True)

# ================================
# 4. Get current position and capture initial image
# ================================
pos = microscope.position
prev_z = pos['z']

#  Capture and save initial (unfocused) image
image = microscope.grab_image()
image_array = np.array(image)
initial_image_path = output_dir / "initial_unfocused_image.png"
Image.fromarray(image_array).save(initial_image_path)
print(f" Initial (unfocused) image saved at: {initial_image_path}")

#  Compute initial variance
gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
variance = cv2.Laplacian(gray, cv2.CV_64F).var()
d_variance = 0.0  # Assuming first run or no previous variance

# ================================
# 5. Predict step size
# ================================
X_sample = np.array([[prev_z, variance, d_variance]])
X_scaled = scaler.transform(X_sample)

step = int(regressor.predict(X_scaled)[0])
print(f"🔁 Predicted step size to move (signed): {step}")

# ================================
# 6. Move and time the process
# ================================
start_time = time.time()

pos['z'] += step
microscope.move(pos)
time.sleep(0.5)

final_image = microscope.grab_image()
end_time = time.time()

# ================================
# 7. Save final (focused) image
# ================================
final_array = np.array(final_image)
final_image_path = output_dir / "final_focused_image.png"
Image.fromarray(final_array).save(final_image_path)

#  Compute final variance
gray_final = cv2.cvtColor(final_array, cv2.COLOR_RGB2GRAY)
variance_final = cv2.Laplacian(gray_final, cv2.CV_64F).var()

# ================================
# 8. Report results
# ================================
elapsed_time = end_time - start_time
print(f" Random Forest-based autofocus complete.")
print(f" Final image saved at: {final_image_path}")
print(f"Time taken to move and capture: {elapsed_time:.2f} seconds")
print(f" Laplacian Variance - Before: {variance:.2f}, After: {variance_final:.2f}")
